In [148]:
from tools import *

# Chiffrement TLWE

In [80]:
def TLWEncrypt(p,q,n,s,sigma,m):
    a=[]
    for i in range(n):
        a.append(randint(0,q-1))
    e = RealDistribution("gaussian",sigma).get_random_element()
    e = round(q*e)
    while(abs(e/q)>=1/(2*p)):
        e = RealDistribution("gaussian",sigma).get_random_element()
        e = round(q*e)
    #print("e= ",e)
    c = AddInTore(q,m*q/p,e)
    for i in range(n):
        c = AddInTore(q,c,MultExtTore(q,s[i],a[i]))
    return a+[c]

In [81]:
def TLWEDecrypt(p,q,n,s,c):
    a = c[:-1]
    m = c[-1]
    for i in range(n):
        m = AddInTore(q,m,MultExtTore(q,-s[i],a[i]))
    return round(m*p/q)%p

In [6]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt = 0
for i in range(1000):
    
    m = randint(0,p-1)


    c = TLWEncrypt(p, q, n, s, sigma, m)

    mmaybe = TLWEDecrypt(p, q, n, s, c)
    if(m==mmaybe):
        cpt = cpt +1
print(cpt)        

1000


## Addition de chiffrés et mutliplication par une constante

In [82]:
def AddCiph(q,n,c1,c2):
    return [AddInTore(q,c1[i],c2[i]) for i in range(n+1)]

In [83]:
def MultCiph(q,n,K,c):
    return [MultExtTore(q,K,c[i]) for i in range(n+1)]

Test : $TLWE_s(m_1) + TLWE_s(m_2) = TLWE_s(m_1+m_2)$

In [9]:
p = 2**9
q = 2**16
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(100):
    
    m1 = randint(0,p-1)
    m2 = randint(0,p-1)

    c1 = TLWEncrypt(p, q, n, s, sigma, m1)
    c2 = TLWEncrypt(p, q, n, s, sigma, m2)

    m = TLWEDecrypt(p, q, n, s, AddCiph(q,n,c1,c2))
    if(m==AddInTore(p,m1,m2)):
        cpt1 = cpt1 +1
print(cpt1)

85


Test : $TLWE_s(m) \cdot K = TLWE_s(m \cdot K)$

In [10]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(100):
    
    m = randint(0,p-1)

    c = TLWEncrypt(p, q, n, s, sigma, m)
    
    K = randint(1,100)
    mmaybe = TLWEDecrypt(p, q, n, s, MultCiph(q,n,K,c))
    
    if(mmaybe==MultExtTore(p,K,m)):
        cpt1 +=1
print(cpt1)

62


## Multiplication de chiffrés : Gadget Matrix

In [84]:
def TrueGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = 1/(B^(j+1))
    return G
#G is the real gadget matrix

In [85]:
def UsedGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = q/(B^(j+1))
    return G
#G is the gadhet matrix of the numerator of the real gadget matrix with q for denominator

In [86]:
def InvGVect(q,B,l,n,v):
    for i in range(len(v)) :
        if v[i]> q/2 :
            v[i] = v[i]-q
    vbar = [round(B**l*v[i]/q) for i in range(len(v))]
    u=[]
    for i in range(len(v)) :
        tmp = (vbar[i]).digits(B)
        while (len(tmp)!=l+1) :
            tmp.append(0)
        for j in range(len(tmp)-1) :
            if tmp[j]>= ceil(B/2) :
                tmp[j] = tmp[j]-B
                tmp[j+1] = tmp[j+1]+1
        tmp.reverse()
        u = u+(tmp[1:])
    return [u]

In [87]:
def InvGMatr(q,B,l,n,M) :
    return [InvGVect(q,B,l,n,M[i]) for i in range(len(M))]

Test : $g^{-1}(v) \cdot G^T = v$ ?

In [88]:
g = InvGVect(64,4,3,1,[41,26])
print(g)
G = UsedGadgetMatrix(64,4,3,1)
MultInMatr(64,g,G)

[[-1, -1, -3, -2, -1, -2]]


[[41, 26]]

## Multiplication de chiffrés : chiffrement TGSW

In [89]:
def TGSWEncrypt(q,B,l,n,m,s):
    sigma = 0.001
    Z = [TLWEncrypt(B**l,q,n,s,sigma,0) for i in range(n+1)]
    return AddInMatr(q,Z,MultExtMatr(q,m,UsedGadgetMatrix(q,B,l,n)))

In [90]:
#c1 et c2 are GSW ciphertexts
def MultInGSW(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGMatr(q,B,l,n,c1),c2)

In [91]:
#c1 is TGSW ciphertext
#c2 is TLWE ciphertext
def MultExtCiphLWE(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGVect(q,B,l,n,c2),c1)

In [19]:
k = 3
m = 5
q = 64
B = 4
l = 2
p = B**l
n = 500
sigma = 2**(-10)
cpt=0
for j in range(10) :
    s = [randint(0,1) for i in range(n)]
    cm = TLWEncrypt(p,q,n,s,sigma,m)

    ck = TGSWEncrypt(q,B,l,n,m,s)
    c = MultExtCiphLWE(q,B,l,n,ck,cm)[0]
    if(TLWEDecrypt(p,q,n,s,c)==MultExtTore(p,k,m)) :
        cpt+=1
print(cpt)


1


# Chiffrement TGLWE

In [92]:
#G^{-1} sur les polynômes
#vX a vector of poly in TQ_N[X]
def InvGVectPoly(q,B,l,N,vX) :
    result = []
    for poly in vX :
        for i in range(len(poly)):
            if poly[i] > q/2 :
                poly[i] = poly[i]-q
        polybar = [round(B**l*poly[i]/q) for i in range(len(poly))]
        u = [[0]*(N) for _ in range(l)]
        for i in range(len(poly)) :
            tmp = polybar[i].digits(B)
            while (len(tmp)!= l+1) :
                tmp.append(0)
            for j in range(len(tmp)-1) :
                if tmp[j] >= ceil(B/2) :
                    tmp[j+1]=tmp[j+1]+1
            tmp.reverse()
            tmp = tmp[1:]
            for index in range(len(tmp)) :
                u[index][i] = u[index][i]+tmp[index]
        result += u
    return [result]

In [93]:
def InvGMatrPoly(q,B,l,N,M) :
    return [InvGVectPoly(q,B,l,N,M[i]) for i in range(len(M))]

In [94]:
def TGLWEncrypt(p,q,N,k,s,sigma,m) :
    a = []
    for i in range(k) :
        a.append([randint(0,q-1) for j in range(N)])
    e = [RealDistribution("gaussian",sigma).get_random_element() for i in range(N)]
    e = [round(q*e[i]) for i in range(N)]
    M = [m[i]*q/p for i in range(N)]
    c = AddPolyInTore(q,N,M,e)
    for i in range(k):
        c = AddPolyInTore(q,N,c,MultPolyExt(q,N,s[i],a[i]))
    return a+[c]

In [120]:
def TGLWEDecrypt(p,q,N,k,s,c) :
    tmp = [0]*N
    for i in range(k) :
        tmp = AddPolyInTore(q,N,tmp,MultPolyExt(q,N,[-(s[i][j]) for j in range(len(s[i]))],c[i]))
    tmp = AddPolyInTore(q,N,tmp,c[len(c)-1])
    result = []
    for i in range(len(tmp)) :
        result.append(round(tmp[i]*p/q)%p)
    return result

In [121]:
#C1 a TGGSW ciphertext (matrix)
#c2 a TGLWE ciphertext (vector)
def MultExtTGLWE(q,N,B,l,C1,c2) :
    return MultMatPoly(q,N,InvGVectPoly(q,B,l,N,c2),C1)

### Chiffrement TGGSW

In [165]:
def TGGSWEncrypt(p,q,B,l,N,k,sigma,s,m):
    Z = [TGLWEncrypt(p,q,N,k,s,sigma,[0]*N) for _ in range((k+1)*l)]
    return AddInMatrPoly(q,N,Z,MultExtMatrPoly(q,N,m,UsedGadgetMatrixPoly(q,B,l,k,N)))

In [166]:
def MultExtMatrPoly(q,N,K,M):
    return [[MultPolyExt(q,N,K,M[i][j]) for j in range(len(M[0]))] for i in range(len(M))]

In [169]:
q =64
p = 16
B = 4
l = 2
k = 2
N = 2
m1 = [1,2]
m2 = [2,1]
M = MultPolyExt(p,N,m1,m2)
print(M)
sigma = 0.001
s = [[0,1],[1,0],[0,0]]
C1 = TGGSWEncrypt(p,q,B,l,N,k,sigma,s,m1)
c2 = TGLWEncrypt(p,q,N,k,s,sigma,m2)
c = MultExtTGLWE(q,N,B,l,C1,c2)[0]
mmaybe = TGLWEDecrypt(p,q,N,k,s,c)
print(mmaybe)

[1, 5]
[6, 7]


# Bootstrapping

In [124]:
def UsedGadgetMatrixPoly(q,B,l,k,N):
    G = [[[0]*N for i in range(k+1)] for j in range((k+1)*l)]
    for i in range(k+1):
        for j in range(0,l):
            G[j+l*i][i][0] = q/(B^(j+1))
    return G

In [125]:
k = 1
N = 2
l = 3
B = 4
q = 256
vX = [[41,26],[231,35]]
g = InvGVectPoly(q,B,l,N,vX)
print(g)
G = UsedGadgetMatrixPoly(q,B,l,k,N)
MultMatPoly(q,N,g,G)

[[[1, 1], [3, 2], [2, 3], [0, 1], [-1, 2], [-2, 1]]]


[[[123, 111], [236, 103]]]

In [131]:
#Cb TGGSW ciphertext
# c0,c1 TGLWE ciphertext
def CMUX(q,N,Cb,c0,c1) : 
    tmp = [AddPolyInTore(q,N,c1[i],[-c0[i][ind] for ind in range(len(c0[i]))]) for i in range(len(c0))]
    tmp2 = MultExtTGLWE(q,N,B,l,Cb,tmp)[0]
    return [AddPolyInTore(q,N,tmp2[i],c0[i]) for i in range(len(c0))]

In [127]:
def loadKeys(p,q,B,l,N,k,s) :
    S = [[randint(0,1) for _ in range(N)] for _ in range(k)]
    Sprime = []
    for si in S :
        Sprime += si
    bs = [TGGSWEncrypt(p,q,B,l,N,k,S, [s[i]] + [0]*(N-1)) for i in range(n)]
    ks = [[_ for j in range(l)] for i in range(k*N)]

    for i in range(k*N) :
        for j in range(l) :
            ks[i][j] = TLWEncrypt(p,q,n,s,sigma,MultExtTore(p,Sprime[i],p/(B**j)))
    return bs,ks

In [128]:
p = 16
q = 64
l = 2
B = 4
n = 5
k = 5
sigma = 0.1
N = 128

s = [1,1,0,1,0]

bs,ks = loadKeys(p,q,B,l,N,k,s)

In [129]:
# c is a TLWE ciphertext : in TQ^{n+1}
# bs = TGGSW_S(s_i)
# ks in TQ^{n+1}
def bootstrapping(p,q,B,l,n,k,N,c,bs,ks) :
    V = [round(p*i/(2*N)) % p for i in range(N)]
    a_tilde = [round(c[i]*2*N/q) % (2*N) for i in range(n)]
    x_tilde = round(c[n]*2*N/q) % (2*N)
    for i in range(n) :
        if (-x_tilde % (2*N) >= N) :
            return "Fail"
    print(a_tilde,x_tilde)
    tmp = [0]*(N)
    tmp[-x_tilde % (2*N)] = 1 # tmp = X^{-x_tilde}
    Q = [[[0]*N for i in range(k)] + [MultPolyExt(q,N,tmp,V)]]
    for i in range(1,n+1):
        tmp = [0]*N # tmp = X^{a_tilde[i]}
        if (a_tilde[i-1]  % (2*N) < N) :
            tmp[a_tilde[i-1] % 2*N] = 1
        else :
            tmp[a_tilde[i-1] % N] = - 1 
        Q.append(CMUX(q,N,bs[i-1],Q[i-1],[MultPolyExt(q,N,tmp,Q[i-1][ind]) for ind in range(len(Q[i-1]))]))
    C = Q[n]
    Cprime = []
    for i in range(k) :
        Cprime += [C[i][0]]
        for j in range(N-1,0,-1) :
            Cprime += [-C[i][j]]
    
    tmp = [0]*(n+1)
    for i in range(k*N) :
        for j in range(l) : 
            tmp2 = [MultExtTore(q,InvGVect(q,B,l,n,[Cprime[i]])[0][j],-ks[i][j][index]) for index in range(n+1)]
            tmp = [AddInTore(q,tmp[index],tmp2[index]) for index in range(n+1)]
    return [AddInTore(q,tmp[index],([0]*(n) + [C[k][0]])[index]) for index in range(n+1)]

In [132]:
while(1) :
    m = randint(0,p-1)
    c = TLWEncrypt(p, q, n, s, sigma, m)
    trux = bootstrapping(p,q,B,l,n,k,N,c,bs,ks)
    if (trux!="Fail") :
        print(TLWEDecrypt(p,q,n,s,trux))
        print("message clair :",m)
        break

[164, 104, 44, 176, 72] 192
10
message clair : 0


In [46]:
trux

[63, 60, 56, 35, 27, 37]

In [48]:
m

4

In [33]:
while(1) :
    m = randint(0,p-1)
    print("Message : ",m)
    c = TLWEncrypt(p, q, n, s, sigma, m)

    K = randint(1,100)
    mmaybe = TLWEDecrypt(p, q, n, s, MultCiph(q,n,K,c))

    if(mmaybe==MultExtTore(p,K,m)):
        print("No need for bootstrapping")
    else :
        print("Need for bootstrapping")
        c_refresh = bootstrapping(p,q,B,l,n,k,N,c,bs,ks)
        mmaybe = TLWEDecrypt(p,q,n,s,c_refresh)
        print(mmaybe)

Message :  3
No need for bootstrapping
Message :  10
No need for bootstrapping
Message :  4
Need for bootstrapping
[204, 204, 48, 32, 136] 244
3
Message :  5
Need for bootstrapping
[156, 228, 88, 116, 148] 72


IndexError: list assignment index out of range